# Wilcoxon Signed-Rank Test
by: *Babak Zolghadr-Asli*

In [ ]:
from scipy.stats import wilcoxon
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from pathlib import Path
import seaborn as sns
import os
from joypy import joyplot
from matplotlib import cm

%matplotlib inline

<div class="alert alert-danger">
    <b>SYNTAX GUIDE</b>
</div>

In order for this script/notebook to work, you need to store the files with a specified name-syntax;

* **Step I.** First all the files in specified folders, where the folder name is as the same as the algorithm name.
* **Step II.** If need be, pass the current direcoty that holds the files to `root_directory` as a `str` object; Note that if you are using a windows base machine you have to use `\` to seprate files location in your directory adress.  
* **Step III.** All results must be stored in a csv file, with the name `'a_fb_result.csv` where `a` is a placeholder for the **algorithm name**, and `b` is the number associated for the function. 



* Note that all these files should have a columns `best_OF`.

In [ ]:
root_directory = 'C:/Users/bz267/Optimization results/Randomness vs Intelligance'

alg_list_temp = os.listdir(root_directory)
alg_list = [i for i in alg_list_temp if not i.startswith(tuple(str(i) for i in range(10)))]
print('=='*30)
print('The follwing algorithms are to be evaluted in this sessin:')
print('=='*30)
for i, alg_name in enumerate(alg_list):
    print('[%d] %s'%((i+1), alg_name))
print('--'*25)

In [ ]:
loc = 'C:/Users/bz267/Optimization results/Randomness vs Intelligance/Elite_Collective_v5/Result/Non-Centered/f1/'
comp_list = os.listdir(loc)
selct_list = [i for i in comp_list if i.startswith('2var')]
nfe_db = pd.DataFrame(columns=['Pop', 'Itr', 'NFE'])
for file_name in selct_list:
    dirct = loc+file_name
    db_temp = pd.read_csv(dirct)
    a, b, c = db_temp['Pop Size'][0], db_temp['Iteration'][0], db_temp['NFE'][0]
    temp_series = pd.DataFrame([a, b, c], index=['Pop', 'Itr', 'NFE']).transpose()
    nfe_db = pd.concat((nfe_db, temp_series), ignore_index=True)
nfe_db.sort_values(['NFE'], inplace=True)
nfe_db.reset_index(inplace=True)
nfe_db.drop(columns=['index'], inplace=True)
nfe_db.head()

<div class="alert alert-success">
    <b>PLOTS</b>
</div>

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=300)

fig = plt.subplot(2,1,1)
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['NFE'], marker='x', mec='r')
fig = plt.grid(alpha=.2)
fig = plt.margins(0.005,.05)
fig = plt.ylabel('NFE values')
# for index in range(len(nfe_db)):
#     fig = plt.text(index-.15, nfe_db['NFE'][index],
#                    '(Pop: %d, Itr: %d)'%(nfe_db['Pop'][index], nfe_db['Itr'][index]),
#                    size='xx-small')

fig = plt.subplot(2,1,2)
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Pop'], label='pop size')
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Itr'], label='max iteration')
fig = plt.grid(alpha=.2)
fig = plt.margins(0.005,.05)
fig = plt.legend(loc='best', framealpha=.5)

fig = plt.tight_layout()

In [ ]:
loc = 'C:/Users/bz267/Optimization results/Randomness vs Intelligance/Elite_Collective_v5/Result/Non-Centered/f1/'
names_2var = [i for i in comp_list if i.startswith('2var')]
names_10var = [i for i in comp_list if i.startswith('10var')]

<div class="alert alert-info">
    <b>I. Non-Centered</b>
</div>

In [ ]:
func_num = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]
pop_values = np.array([100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])

In [ ]:
# RAEDING AND CREATING FILES WITH OFS OF ALL FUNCTIONS
columns = ['Pop', 'Itr', 'NFE', 'Min', 'Q25', 'Q50', 'Q75', 'Max', 'Avr', 'Std']
for num_var in num_variables_values:
    full_name_list = eval('names_%dvar'%num_var)
    for f in func_num:
        for alg_name in alg_list:
            globals()['nc_%dvar_f%d_%s'%(num_var, f, alg_name)] = pd.DataFrame(columns=columns)
            for file_name in full_name_list:
                folder_name = '/%s/Result/Non-Centered/f%d/'%(alg_name, f)
                db = pd.read_csv(root_directory+folder_name+file_name)
                q1, q2, q3 = db['best_OF'].quantile(0.25), db['best_OF'].quantile(0.5), db['best_OF'].quantile(0.75)
                q0, q4 = db['best_OF'].min(), db['best_OF'].max()
                mean, std = db['best_OF'].mean(), db['best_OF'].std()
                pop, it, NFE = db['Pop Size'][0], db['Iteration'][0], db['NFE'][0]
                temp_db = pd.DataFrame([pop, it, NFE, q0, q1, q2, q3, q4, mean, std], index=columns).transpose()
                exec('nc_%dvar_f%d_%s = pd.concat((nc_%dvar_f%d_%s, temp_db), ignore_index=True)'%(num_var, f, alg_name, num_var, f, alg_name))
            eval("nc_%dvar_f%d_%s.sort_values(['NFE'], inplace=True)"%(num_var, f, alg_name,))

In [ ]:
num_var = 10 #2

alg_name = 'Elite_Greedy_Collective_v5'#'Pure_Random_Greedy' #'Elite_Greedy_Collective_v1'#'Elite_Greedy_Collective_v5'#'Elite_Collective_v5'#'Elite_Collective_v1'

func_num = 4
plot_db = eval("nc_%dvar_f%d_%s"%(num_var, func_num, alg_name))

In [ ]:
fig = plt.figure(figsize=(12, 7), dpi=300)

fig = plt.subplot(2,1,1)
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
fig = plt.xlabel('NFE')
fig = plt.ylabel(r'$f(x)$')
fig = plt.margins(0.005,.05)
fig = plt.grid(alpha=.5)
fig = plt.legend(loc='best', framealpha=.5)

fig = plt.subplot(2,1,2)
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Pop'], marker='x', mec='r', label='Population size')
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Itr'], marker='x', mec='r', label='Iteration')
fig = plt.grid(alpha=.2)
fig = plt.margins(0.005,.05)
fig = plt.legend(loc='best', framealpha=.5)

fig = plt.tight_layout()

In [ ]:
plot_db = nc_10var_f10_Elite_Collective_v1

fig = plt.figure(figsize=(12, 3), dpi=300)
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
fig = plt.xlabel('NFE')
fig = plt.ylabel(r'$f(x)$')
fig = plt.margins(0.005,.05)
fig = plt.grid(alpha=.5)
fig = plt.legend(loc='best', framealpha=.5, ncol=2)
fig = plt.show()
# fig = plt.savefig(output_dir / file_name)
fig = plt.close()

In [ ]:
for num_var in num_variables_values:
    for alg_name in alg_list:  
        for f in func_num:
            folder_name = '/0Summary result/Plots/NC/Convergance Plot/%d_vars/%s/'%(num_var, alg_name)
            file_name = 'f%d_conv_plot.png'%(f)
            output_dir = Path(root_directory+folder_name)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            plot_db = eval("nc_%dvar_f%d_%s"%(num_var, f, alg_name))
            
            fig = plt.figure(figsize=(12, 3), dpi=300)
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
            fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
            fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
            fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
            fig = plt.xlabel('NFE')
            fig = plt.ylabel(r'$f(x)$')
            fig = plt.margins(0.005,.05)
            fig = plt.grid(alpha=.5)
            fig = plt.legend(loc='best', framealpha=.5, ncol=2)
            fig = plt.tight_layout()
            fig = plt.savefig(output_dir / file_name)
            fig = plt.show()

In [ ]:
alg_list = ['Pure_Random', 'Pure_Random_Greedy',
            'Elite_Collective_v5', 'Elite_Collective_v1',
           'Elite_Greedy_Collective_v5', 'Elite_Greedy_Collective_v1']
alg_list

In [ ]:
for num_var in num_variables_values:
    folder_name = '/0Summary result/Plots/NC/Comparison Convergance Plot/%d_vars/'%(num_var)
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    for f in func_num:
        file_name = 'f%d_conv_comp_plot.png'%(f)
        fig = plt.figure(figsize=(12, 12), dpi=300)
        for index, alg_name in enumerate(alg_list):
            plot_db = eval("nc_%dvar_f%d_%s"%(num_var, f, alg_name))
            fig = plt.subplot(6, 1, index+1)
            print(alg_name)
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
            fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
            fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
            fig = plt.ylabel(r'$f(x)$')
            fig = plt.margins(0.005,.05)
            fig = plt.grid(alpha=.5)
            fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
            if index==5:
                fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
                fig = plt.xlabel('NFE')
                fig = plt.legend(loc='best', framealpha=.5, ncol=2)
            else:
                ax = plt.gca()
                ax.set_xticklabels([])
        fig = plt.tight_layout()
        fig = plt.savefig(output_dir / file_name)
        fig = plt.show()

<div class="alert alert-info">
    <b>II. Centered</b>
</div>

In [ ]:
func_num = [1,2,4,6,7]
pop_values = np.array([100, 200, 500, 750, 1000])
iteration_values = np.array([500, 750, 1000, 2000, 2500])
num_variables_values = np.array([2, 10])

In [ ]:
# RAEDING AND CREATING FILES WITH OFS OF ALL FUNCTIONS
columns = ['Pop', 'Itr', 'NFE', 'Min', 'Q25', 'Q50', 'Q75', 'Max', 'Avr', 'Std']
for num_var in num_variables_values:
    full_name_list = eval('names_%dvar'%num_var)
    for f in func_num:
        for alg_name in alg_list:
            globals()['c_%dvar_f%d_%s'%(num_var, f, alg_name)] = pd.DataFrame(columns=columns)
            for file_name in full_name_list:
                folder_name = '/%s/Result/Centered/f%d/'%(alg_name, f)
                db = pd.read_csv(root_directory+folder_name+file_name)
                q1, q2, q3 = db['best_OF'].quantile(0.25), db['best_OF'].quantile(0.5), db['best_OF'].quantile(0.75)
                q0, q4 = db['best_OF'].min(), db['best_OF'].max()
                mean, std = db['best_OF'].mean(), db['best_OF'].std()
                pop, it, NFE = db['Pop Size'][0], db['Iteration'][0], db['NFE'][0]
                temp_db = pd.DataFrame([pop, it, NFE, q0, q1, q2, q3, q4, mean, std], index=columns).transpose()
                exec('c_%dvar_f%d_%s = pd.concat((c_%dvar_f%d_%s, temp_db), ignore_index=True)'%(num_var, f, alg_name, num_var, f, alg_name))
            eval("c_%dvar_f%d_%s.sort_values(['NFE'], inplace=True)"%(num_var, f, alg_name,))

In [ ]:
num_var = 10 #2

alg_name = 'Pure_Random'#'Pure_Random_Greedy' #'Elite_Greedy_Collective_v1'#'Elite_Greedy_Collective_v5'#'Elite_Collective_v5'#'Elite_Collective_v1'

func_num = 4
plot_db = eval("c_%dvar_f%d_%s"%(num_var, func_num, alg_name))

In [ ]:
fig = plt.figure(figsize=(12, 7), dpi=300)

fig = plt.subplot(2,1,1)
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
fig = plt.xlabel('NFE')
fig = plt.ylabel(r'$f(x)$')
fig = plt.margins(0.005,.05)
fig = plt.grid(alpha=.5)
fig = plt.legend(loc='best', framealpha=.5)

fig = plt.subplot(2,1,2)
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Pop'], marker='x', mec='r', label='Population size')
fig = plt.plot(np.arange(len(nfe_db)),nfe_db['Itr'], marker='x', mec='r', label='Iteration')
fig = plt.grid(alpha=.2)
fig = plt.margins(0.005,.05)
fig = plt.legend(loc='best', framealpha=.5)

fig = plt.tight_layout()

In [ ]:
plot_db = c_10var_f4_Elite_Greedy_Collective_v5

fig = plt.figure(figsize=(12, 3), dpi=300)
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
fig = plt.xlabel('NFE')
fig = plt.ylabel(r'$f(x)$')
fig = plt.margins(0.005,.05)
fig = plt.grid(alpha=.5)
fig = plt.legend(loc='best', framealpha=.5, ncol=2)
fig = plt.show()
# fig = plt.savefig(output_dir / file_name)
fig = plt.close()

In [ ]:
for num_var in num_variables_values:
    for alg_name in alg_list:  
        for f in func_num:
            folder_name = '/0Summary result/Plots/C/Convergance Plot/%d_vars/%s/'%(num_var, alg_name)
            file_name = 'f%d_conv_plot.png'%(f)
            output_dir = Path(root_directory+folder_name)
            output_dir.mkdir(parents=True, exist_ok=True)
            
            plot_db = eval("c_%dvar_f%d_%s"%(num_var, f, alg_name))
            
            fig = plt.figure(figsize=(12, 3), dpi=300)
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
            fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
            fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
            fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
            fig = plt.xlabel('NFE')
            fig = plt.ylabel(r'$f(x)$')
            fig = plt.margins(0.005,.05)
            fig = plt.grid(alpha=.5)
            fig = plt.legend(loc='best', framealpha=.5, ncol=2)
            fig = plt.tight_layout()
            fig = plt.savefig(output_dir / file_name)
            fig = plt.show()

In [ ]:
alg_list = ['Pure_Random', 'Pure_Random_Greedy',
            'Elite_Collective_v5', 'Elite_Collective_v1',
           'Elite_Greedy_Collective_v5', 'Elite_Greedy_Collective_v1']
alg_list

In [ ]:
for index, alg_name in enumerate(alg_list):
    print(alg_name)

In [ ]:
f, num_var = 4, 10

file_name = 'f%d_conv_comp_plot.png'%(f)
fig = plt.figure(figsize=(12, 12), dpi=300)
for index, alg_name in enumerate(alg_list):
    plot_db = eval("c_%dvar_f%d_%s"%(num_var, f, alg_name))
    fig = plt.subplot(6, 1, index+1)
#     print(alg_name)
#     fig = plt.title('%s'%alg_name)
    fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
    fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
    fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
    fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
    fig = plt.ylabel(r'$f(x)$')
    fig = plt.margins(0.005,.05)
    fig = plt.grid(alpha=.5)
    fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
    if index==5:
        fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
        fig = plt.xlabel('NFE')
        fig = plt.legend(loc='best', framealpha=.5, ncol=2)
    else:
        ax = plt.gca()
        ax.set_xticklabels([])
fig = plt.tight_layout()
fig = plt.show()

In [ ]:
for num_var in num_variables_values:
    folder_name = '/0Summary result/Plots/C/Comparison Convergance Plot/%d_vars/'%(num_var)
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    for f in func_num:
        file_name = 'f%d_conv_comp_plot.png'%(f)
        fig = plt.figure(figsize=(12, 12), dpi=300)
        for index, alg_name in enumerate(alg_list):
            plot_db = eval("c_%dvar_f%d_%s"%(num_var, f, alg_name))
            fig = plt.subplot(6, 1, index+1)
            print(alg_name)
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Min'], plot_db['Max'], color='r', alpha=.1, label='Range')
            fig = plt.fill_between(np.arange(len(plot_db)), plot_db['Q25'], plot_db['Q75'], color='g', alpha=.2, label='IQR')
            fig = plt.plot(np.arange(len(plot_db)), plot_db['Q50'], ls=':', c='b', lw=.75, label='Median')
            fig = plt.scatter(np.arange(len(plot_db)), plot_db['Avr'], marker='x', c='r', s=10, label='Average')
            fig = plt.ylabel(r'$f(x)$')
            fig = plt.margins(0.005,.05)
            fig = plt.grid(alpha=.5)
            fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
            if index==5:
                fig = plt.xticks(ticks=np.arange(len(plot_db)), labels=['%.1E'%i for i in plot_db['NFE']], rotation=90)
                fig = plt.xlabel('NFE')
                fig = plt.legend(loc='best', framealpha=.5, ncol=2)
            else:
                ax = plt.gca()
                ax.set_xticklabels([])
        fig = plt.tight_layout()
        fig = plt.savefig(output_dir / file_name)
        fig = plt.show()

<div class="alert alert-warning">
    <b>Analysis</b>
</div>

MAX NFE / TARGET LAST OPTION ELITE GREEDY V1

##### NC

In [ ]:
func_num = [1,2,3,4,5,6,7,8,9,10,21,23,24,25,26,27,28]
alg_list = ['Pure_Random', 'Pure_Random_Greedy',
            'Elite_Collective_v5', 'Elite_Collective_v1',
           'Elite_Greedy_Collective_v5', 'Elite_Greedy_Collective_v1']
pop_val = 1000
it_val = 2500
num_variables_values = np.array([2, 10])

In [ ]:
# RAEDING AND CREATING FILES WITH OFS OF ALL FUNCTIONS
for num_var in num_variables_values:
    for f in func_num:
        globals()['overall_nc_%dvar_f%d'%(num_var, f)] = pd.DataFrame(columns=alg_list)
        file_name = '%dvar_%dpop_%dit.csv'%(num_var, pop_val, it_val)
        for alg_name in alg_list:
            folder_name = '/%s/Result/Non-Centered/f%d/'%(alg_name, f)
            db = pd.read_csv(root_directory+folder_name+file_name)
            exec("overall_nc_%dvar_f%d[alg_name] = db['best_OF']"%(num_var, f))
        exec("overall_nc_%dvar_f%d.dropna(inplace=True)"%(num_var, f))

In [ ]:
target = alg_list[0]
confidence_level = .05

for num_var in num_variables_values:
    headers = eval('overall_nc_%dvar_f%d.columns'%(num_var, f))
    rest_name = [i for i in headers if i != target]
    exec('%s_%dvar_wilcox = pd.DataFrame(columns=rest_name)'%(target, num_var))
    for f in func_num:
        results = list()
        target_val = eval('overall_nc_%dvar_f%d[target]'%(num_var, f))
        for alg in rest_name:
            diff = target_val - eval('overall_nc_%dvar_f%d["%s"]'%(num_var, f, alg))
            w, p = wilcoxon(diff, zero_method="zsplit",  method='approx')
            if not (p<confidence_level):
                symbol = '0'
            else:
                w, p = wilcoxon(diff, zero_method="zsplit", alternative='less',  method='approx')
                if p<confidence_level:
                    symbol = '-'
                else:
                    symbol = '+'
            results.append(symbol)  
        exec('%s_%dvar_wilcox=pd.concat((%s_%dvar_wilcox, pd.DataFrame(results, index=rest_name).transpose()), ignore_index=True)'
             %(target, num_var, target, num_var))
    index = ['f%d'%i for i in func_num]
    eval("%s_%dvar_wilcox.insert(0,'Function',index)"%(target, num_var))
    folder_name = '/0Summary result/Stat tests/NC/Wilcox/%dvar/'%num_var
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    file_name = '%s_wilcox.csv'%target
    exec("%s_%dvar_wilcox.to_csv(output_dir / file_name)"%(target, num_var))

##### C

In [ ]:
func_num = [1,2,4,6,7]
alg_list = ['Pure_Random', 'Pure_Random_Greedy',
            'Elite_Collective_v5', 'Elite_Collective_v1',
           'Elite_Greedy_Collective_v5', 'Elite_Greedy_Collective_v1']
pop_val = 1000
it_val = 2500
num_variables_values = np.array([2, 10])

In [ ]:
# RAEDING AND CREATING FILES WITH OFS OF ALL FUNCTIONS
for num_var in num_variables_values:
    for f in func_num:
        globals()['overall_c_%dvar_f%d'%(num_var, f)] = pd.DataFrame(columns=alg_list)
        file_name = '%dvar_%dpop_%dit.csv'%(num_var, pop_val, it_val)
        for alg_name in alg_list:
            folder_name = '/%s/Result/Centered/f%d/'%(alg_name, f)
            db = pd.read_csv(root_directory+folder_name+file_name)
            exec("overall_c_%dvar_f%d[alg_name] = db['best_OF']"%(num_var, f))
        exec("overall_c_%dvar_f%d.dropna(inplace=True)"%(num_var, f))

In [ ]:
target = alg_list[5]
confidence_level = .05

for num_var in num_variables_values:
    headers = eval('overall_c_%dvar_f%d.columns'%(num_var, f))
    rest_name = [i for i in headers if i != target]
    exec('%s_%dvar_wilcox = pd.DataFrame(columns=rest_name)'%(target, num_var))
    for f in func_num:
        results = list()
        target_val = eval('overall_c_%dvar_f%d[target]'%(num_var, f))
        for alg in rest_name:
            diff = target_val - eval('overall_c_%dvar_f%d["%s"]'%(num_var, f, alg))
            w, p = wilcoxon(diff, zero_method="zsplit",  method='approx')
            if not (p<confidence_level):
                symbol = '0'
            else:
                w, p = wilcoxon(diff, zero_method="zsplit", alternative='less',  method='approx')
                if p<confidence_level:
                    symbol = '-'
                else:
                    symbol = '+'
            results.append(symbol)  
        exec('%s_%dvar_wilcox=pd.concat((%s_%dvar_wilcox, pd.DataFrame(results, index=rest_name).transpose()), ignore_index=True)'
             %(target, num_var, target, num_var))
    index = ['f%d'%i for i in func_num]
    eval("%s_%dvar_wilcox.insert(0,'Function',index)"%(target, num_var))
    folder_name = '/0Summary result/Stat tests/C/Wilcox/%dvar/'%num_var
    output_dir = Path(root_directory+folder_name)
    output_dir.mkdir(parents=True, exist_ok=True)
    file_name = '%s_wilcox.csv'%target
    exec("%s_%dvar_wilcox.to_csv(output_dir / file_name)"%(target, num_var))